# Packages

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from gensim.similarities.annoy import AnnoyIndexer

from utils import *

import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from tqdm import tqdm
import polars as pl
from annoy import AnnoyIndex
import polars as pl
import polars as pl
from utils import *
from src.eval import model_eval
from src.config import (
    raw_data_session_id_dir, candidate_dir, model_for_eval, candidate_file_name, submit_file_name
    # , w2v_model_file_name, w2v_index_file_name
)


# Config

In [2]:
debug = False

topn = 100
model_version = 'w2v_v3'
if debug:
    n_rows = 100
else:
    n_rows = None

    
task = 'task1'
# Common setting
model_dir = f'../model_training/{model_version}/'
# target locales: locales needed for task1
target_locals = ["DE", 'JP', 'UK']
submit_file = os.path.join('../data/sub_files/', 
                           submit_file_name.format(
                                task=task
                               , model_version=model_version
                               , model_for_eval=model_for_eval
                               , topn=topn
                           )
                          )

train_cg_file = os.path.join(base_dir,
                             candidate_dir, 
            candidate_file_name.format(
                task=task
                , data_type='train'
                , model_version=model_version
                , model_for_eval=model_for_eval
                , topn=topn
            )
                            )
eval_cg_file = os.path.join(base_dir,
                            candidate_dir, 
            candidate_file_name.format(
                task=task
                , data_type='eval'
                , model_version=model_version
                , model_for_eval=model_for_eval
                , topn=topn
            )
                            )
test_cg_file = os.path.join(base_dir,
                            candidate_dir, 
            candidate_file_name.format(
                task=task
                , data_type='test'
                , model_version=model_version
                , model_for_eval=model_for_eval
                , topn=topn
            )
                            )

# specific setting
num_tree = 100
# if model_for_eval:
w2v_model_file = os.path.join(model_dir, f"{model_for_eval}.model")
annoy_index_file = os.path.join(model_dir, f"{str(num_tree)}_{model_for_eval}.index")

In [3]:
# submit_file_name

In [4]:
! mkdir {model_dir}

mkdir: cannot create directory ‘../model_training/w2v_v3/’: File exists


In [5]:
print(f"{w2v_model_file}; \n {annoy_index_file}; \n {submit_file}  \n {train_cg_file} \n {eval_cg_file} \n {test_cg_file}")

../model_training/w2v_v3/True.model; 
 ../model_training/w2v_v3/100_True.index; 
 ../data/sub_files/submit_task1_w2v_v3_True_top100.parquet  
 ../data/candidates/task1_train_w2v_v3_True_top100.parquet 
 ../data/candidates/task1_eval_w2v_v3_True_top100.parquet 
 ../data/candidates/task1_test_w2v_v3_True_top100.parquet


# Function 

In [6]:
def w2v_rec(target_df, w2vec, annoy_index, topn=100):
    final_cols = ['session_id', 'next_item_prediction']
    target_df = (
        target_df.with_columns(
            pl.col('prev_items').apply(lambda x: 
                                       # x.to_list()
                                       list(map(list, zip(*w2vec.wv.most_similar(positive=x.to_list(),
                                                                     topn=topn,
                                                                     indexer=annoy_index))))[0]
                                      ).alias('next_item_prediction'))
        .select(
          final_cols
        )
    )
    return target_df

# Word2vec

## Train data process

In [7]:
train_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_train.parquet'), n_rows=n_rows).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

eval_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'), n_rows=n_rows).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

# df_sess.head(3).collect()
test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1.parquet'), n_rows=n_rows).with_columns(pl.col('prev_items').apply(str2list))


## Train model & annnoy index 

In [8]:
train_data = train_pl.with_columns(
    pl.col('prev_items').arr.concat(pl.col('next_item')) 
)
if not model_for_eval:
    eval_data = eval_pl.with_columns(
    pl.col('prev_items').arr.concat(pl.col('next_item')) 
)
else:
    eval_data = eval_pl
test_data = test_pl

In [9]:
cols_to_keep = ['prev_items']
sentences = (
    pl.concat([train_data.select(cols_to_keep), eval_data.select(cols_to_keep), test_data.select(cols_to_keep)], how='vertical')
).collect().to_dict()['prev_items'].to_list()

# sentences = map(sentences, list)


len(sentences)

3589687

In [10]:
# sentences

In [11]:
# sentences

In [12]:
vector_size = 32
epochs = 10
sg = 1 # 1 for skip-gram
pop_thresh = 0.82415
window = 4

# 

# len(sentences)sentences

In [13]:
from gensim.models.callbacks import CallbackAny2Vec

class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0
        self.loss_to_be_subed = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss_now = loss - self.loss_to_be_subed
        self.loss_to_be_subed = loss
        print('Loss after epoch {}: {}'.format(self.epoch, loss_now))
        self.epoch += 1

In [14]:
w2vec = Word2Vec(sentences=sentences, vector_size=vector_size, epochs = epochs, sg=sg,
                 min_count=1, workers=14,
                 window=window,
                  compute_loss=True
              , callbacks=[callback()])

Loss after epoch 0: 14782675.0
Loss after epoch 1: 5239365.0
Loss after epoch 2: 3863152.0
Loss after epoch 3: 3312670.0
Loss after epoch 4: 2841758.0
Loss after epoch 5: 2481642.0
Loss after epoch 6: 1343122.0
Loss after epoch 7: 544188.0
Loss after epoch 8: 510712.0
Loss after epoch 9: 483772.0


In [15]:
w2v_model_file

'../model_training/w2v_v3/True.model'

In [16]:

# 100 trees are being used in this example
annoy_index = AnnoyIndexer(w2vec, num_tree)

## Save Model 

In [17]:
w2vec.save(w2v_model_file)

In [18]:

annoy_index.save(annoy_index_file)

## Load model 

In [19]:
w2v_model_file

'../model_training/w2v_v3/True.model'

In [20]:
annoy_index_file

'../model_training/w2v_v3/100_True.index'

In [21]:
w2vec = Word2Vec.load(w2v_model_file)
annoy_index = AnnoyIndexer()
annoy_index.load(annoy_index_file)

## Model Eval

In [22]:
eval_candidate_pl = w2v_rec(target_df=eval_pl,
                   w2vec=w2vec, annoy_index=annoy_index, topn=100)

In [23]:
eval_candidate_pl.schema

{'session_id': Int64, 'next_item_prediction': Unknown}

In [24]:
%%time
eval_candidate_pl = eval_candidate_pl.collect()

CPU times: user 14min 16s, sys: 6.72 s, total: 14min 23s
Wall time: 14min 23s


In [25]:
model_eval(target_df=eval_pl.join(eval_candidate_pl.lazy(), how='left', on='session_id'))

total_sessions,mrr,recall@20,recall@100
u32,f64,f64,f64
326443,0.046979,0.195979,0.338405


## Candidate Saving 

In [26]:
test_candidate_pl  = w2v_rec(target_df=test_pl,
                   w2vec=w2vec, annoy_index=annoy_index, topn=100)
train_candidate_pl  = w2v_rec(target_df=train_pl,
                   w2vec=w2vec, annoy_index=annoy_index, topn=100)

In [27]:
%%time
train_candidate_pl = train_candidate_pl.collect()

CPU times: user 1h 59min 55s, sys: 53.8 s, total: 2h 49s
Wall time: 2h 51s


In [28]:
%%time
test_candidate_pl = test_candidate_pl.collect()

CPU times: user 11min 37s, sys: 4.15 s, total: 11min 41s
Wall time: 11min 41s


In [39]:
train_candidate_pl.head()

session_id,next_item_prediction
i64,list[str]
0,"[""B09JSPLN1M"", ""B09W9FND7K"", … ""B088TJJQHG""]"
1,"[""B08D9SVRFS"", ""B084WZYCW3"", … ""B06ZYCTQ3S""]"
2,"[""B0B1LGXWDS"", ""B00AZYORS2"", … ""B092JDH72P""]"
5,"[""B0749RNHRY"", ""B0749V8TC7"", … ""B07T22XJG2""]"
6,"[""B09SMK3R8H"", ""B01N4ND0F9"", … ""B07HTMYH43""]"


### Saving 

In [29]:
eval_candidate_pl.write_parquet(eval_cg_file)
train_candidate_pl.write_parquet(train_cg_file)
test_candidate_pl.write_parquet(test_cg_file)

In [30]:
# df = test_sessions.sample(10)

In [31]:
# def i2i(df):
#     pl_df = pl.from_dataframe(df)
#     pl_df = (
#         pl_df
#             .with_columns(pl.col('prev_items').apply(lambda row: get_rec(row, annoy_index=annoy_index, topn=100)).alias('next_item_prediction'))
#     )
#     return pl_df

In [32]:
# # %%time
# def pd_get_rec(df, w2vec, annoy_index, topn):
#     next_item_prediction_lst = []
#     for a in tqdm(df.iterrows(), total=len(df)):
#         prev_items = a[1]['prev_items']
#         res = [ele.replace('[', '').replace(']', '').replace('\n', '').replace("'", '').replace(' ', '') for ele in prev_items.split(' ')]
#         # print(type(res))
#         similarity_dic = w2vec.wv.most_similar(positive=res, topn=topn, indexer=annoy_index)
#         res = [item for item, simi in similarity_dic]
#         next_item_prediction_lst.append(res)
#     df['next_item_prediction'] = next_item_prediction_lst
#     return df

# Validate predictions ✅ 😄

In [33]:
predictions = test_pl.join(test_candidate_pl.lazy(), how='left', on='session_id').collect()[['locale', 'next_item_prediction']].to_pandas()

In [34]:
check_predictions(predictions, test_sessions=test_pl.collect().to_pandas(), 
                  # check_products=True, product_df=products
                 )

In [35]:
# Its important that the parquet file you submit is saved with pyarrow backend
predictions.to_parquet(submit_file, engine='pyarrow')

In [36]:
submit_file

'../data/sub_files/submit_task1_w2v_v3_True_top100.parquet'

## Submit to AIcrowd 🚀

In [37]:
# # You can submit with aicrowd-cli, or upload manually on the challenge page.
# !aicrowd submission create -c task-1-next-product-recommendation -f {submit_file}